In [ ]:
%matplotlib inline
import os
import bmp280_log as log

In [ ]:
# settings
timeout = 5
path = os.path.join('..', 'data')
prefix = 'bmp280_obs'

# run the logger, it will listen as long as the Arduino provides correctly-formatted data, 
# is connected and duration [sec] is not reached yet
df = log.log_bmp280(path, prefix, duration=1000, baud_rate=115200)
df


In [ ]:
df.describe()

In [ ]:
import pandas as pd
pd.plotting.autocorrelation_plot(df['pressure [Pa]'])

In [ ]:
plt.plot(df['temperature [oC]'].values, df['pressure [Pa]'].values, 'o', color='k', alpha=0.2)